## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-14-56-35 +0000,startribune,The latest: Judge to hold hearing on ICE surge...,https://www.startribune.com/ice-raids-minnesot...
1,2026-01-26-14-53-00 +0000,wsj,"Lands’ End, WHP Global Form Joint Venture",https://www.wsj.com/finance/investing/lands-en...
2,2026-01-26-14-48-57 +0000,nyt,Israel Says It Will Reopen Gaza-Egypt Border C...,https://www.nytimes.com/2026/01/26/world/middl...
3,2026-01-26-14-47-54 +0000,nyt,Airlines Cancel Hundreds of Flights In Anticip...,https://www.nytimes.com/2026/01/23/business/ai...
4,2026-01-26-14-47-47 +0000,nyt,Winter Storm Dumps a Foot of Snow in at Least ...,https://www.nytimes.com/2026/01/25/us/snow-tot...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2449/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
56,trump,34
9,shooting,23
261,minneapolis,22
6,minnesota,21
33,storm,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
174,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...,131
60,2026-01-26-12-22-24 +0000,nypost,Powerful House Republican suggests Trump consi...,https://nypost.com/2026/01/26/us-news/powerful...,126
124,2026-01-26-09-46-21 +0000,bbc,Trump says administration 'reviewing everythin...,https://www.bbc.com/news/articles/cr571qg4m61o...,98
66,2026-01-26-12-03-00 +0000,wsj,Republican unity over President Trump’s immigr...,https://www.wsj.com/politics/policy/miller-noe...,95
201,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
174,131,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...
162,57,2026-01-26-04-30-19 +0000,nypost,Violent anti-ICE protesters swarm Minnesota ho...,https://nypost.com/2026/01/25/us-news/anti-ice...
135,53,2026-01-26-07-47-57 +0000,nypost,Missing Kansas teacher found dead in snow afte...,https://nypost.com/2026/01/26/us-news/missing-...
201,42,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
54,33,2026-01-26-12-46-55 +0000,nyt,"In Xi’s China, Top General’s Fall Shows Precar...",https://www.nytimes.com/2026/01/26/world/asia/...
64,32,2026-01-26-12-10-00 +0000,wsj,Allied Gold Agreed to $4 Billion Buyout By Hon...,https://www.wsj.com/business/allied-gold-agree...
153,29,2026-01-26-05-06-32 +0000,nypost,Crazy scheme saw San Francisco taxpayers shell...,https://nypost.com/2026/01/26/us-news/san-fran...
238,28,2026-01-25-19-20-05 +0000,nypost,"Hochul demands removal of Noem, Border Patrol ...",https://nypost.com/2026/01/25/us-news/hochul-d...
21,27,2026-01-26-14-25-51 +0000,nypost,Body of final Israeli hostage Ran Gvili recove...,https://nypost.com/2026/01/26/world-news/body-...
47,27,2026-01-26-13-04-50 +0000,nypost,Dems need to ‘tell celebrities to just shut th...,https://nypost.com/2026/01/26/us-news/dems-sho...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
